Scripts for regression experiments on mouse

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import mp_run


%load_ext autoreload
%autoreload 2
from multiprocessing import Pool, cpu_count


# regex for number extraction from string
number_pattern =  r'(-?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+-]?\d+)?)'


In [2]:
ref_res_df = pd.read_csv('../output/network_model/yeast_all_tf_high_var_target_efron_train.csv.gz', compression='gzip', index_col=0)

In [3]:
train_source_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/train_source.csv.gz', compression='gzip', index_col=0)
train_target_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/train_target.csv.gz', compression='gzip', index_col=0)
test_source_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/test_source.csv.gz', compression='gzip', index_col=0)
test_target_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/test_target.csv.gz', compression='gzip', index_col=0)

train_source_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/train_source.csv.gz', compression='gzip', index_col=0)
train_target_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/train_target.csv.gz', compression='gzip', index_col=0)
test_source_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/test_source.csv.gz', compression='gzip', index_col=0)
test_target_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/test_target.csv.gz', compression='gzip', index_col=0)

train_source_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/train_source.csv.gz', compression='gzip', index_col=0)
train_target_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/train_target.csv.gz', compression='gzip', index_col=0)
test_source_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/test_source.csv.gz', compression='gzip', index_col=0)
test_target_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/test_target.csv.gz', compression='gzip', index_col=0)

In [4]:
train_source_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_1.csv.gz', compression='gzip', index_col=0)
train_target_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_1.csv.gz', compression='gzip', index_col=0)
test_source_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_1.csv.gz', compression='gzip', index_col=0)
test_target_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_1.csv.gz', compression='gzip', index_col=0)

train_source_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_2.csv.gz', compression='gzip', index_col=0)
train_target_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_2.csv.gz', compression='gzip', index_col=0)
test_source_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_2.csv.gz', compression='gzip', index_col=0)
test_target_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_2.csv.gz', compression='gzip', index_col=0)

train_source_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_3.csv.gz', compression='gzip', index_col=0)
train_target_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_3.csv.gz', compression='gzip', index_col=0)
test_source_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_3.csv.gz', compression='gzip', index_col=0)
test_target_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_3.csv.gz', compression='gzip', index_col=0)

train_source_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_4.csv.gz', compression='gzip', index_col=0)
train_target_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_4.csv.gz', compression='gzip', index_col=0)
test_source_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_4.csv.gz', compression='gzip', index_col=0)
test_target_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_4.csv.gz', compression='gzip', index_col=0)

train_source_df_4 = pd.concat([train_source_4_1, train_source_4_2, train_source_4_3, train_source_4_4], axis=1)
train_target_df_4 = pd.concat([train_target_4_1, train_target_4_2, train_target_4_3, train_target_4_4], axis=1)
test_source_df_4 = pd.concat([test_source_4_1, test_source_4_2, test_source_4_3, test_source_4_4], axis=1)
test_target_df_4 = pd.concat([test_target_4_1, test_target_4_2, test_target_4_3, test_target_4_4], axis=1)


In [5]:
train_source = pd.concat([train_source_df_1, train_source_df_2, train_source_df_3, train_source_4_1, train_source_4_2, train_source_4_3, train_source_4_4], axis=1)
train_target = pd.concat([train_target_df_1, train_target_df_2, train_target_df_3, train_target_4_1, train_target_4_2, train_target_4_3, train_target_4_4], axis=1)
test_source = pd.concat([test_source_df_1, test_source_df_2, test_source_df_3, test_source_4_1, test_source_4_2, test_source_4_3, test_source_4_4], axis=1)
test_target = pd.concat([test_target_df_1, test_target_df_2, test_target_df_3, test_target_4_1, test_target_4_2, test_target_4_3, test_target_4_4], axis=1)

common_genes = list(train_source.index)

In [6]:
train_source_list = [train_source_df_1, train_source_df_2, train_source_df_3, train_source_df_4]
train_target_list = [train_target_df_1, train_target_df_2, train_target_df_3, train_target_df_4]
test_source_list = [test_source_df_1, test_source_df_2, test_source_df_3, test_source_df_4]
test_target_list = [test_target_df_1, test_target_df_2, test_target_df_3, test_target_df_4]

alt_train_source_list = [train_source_4_1, train_source_4_2, train_source_4_3, train_source_4_4]
alt_train_target_list = [train_target_4_1, train_target_4_2, train_target_4_3, train_target_4_4]
alt_test_source_list = [test_source_4_1, test_source_4_2, test_source_4_3, test_source_4_4]
alt_test_target_list = [test_target_4_1, test_target_4_2, test_target_4_3, test_target_4_4]

In [7]:
# get network data, training features
tf_set = set()
tf_list_df = pd.read_csv('../data/yeast/yeast_tf_list.tsv.gz', sep='\t', compression='gzip', index_col=0)
for name in tf_list_df['Gene Names']:
    name_splits = name.split(' ')
    for i in name_splits:
        if i.upper() in train_source.index:
            tf_set.add(i.upper())

network_df = pd.read_csv('../data/yeast/yeat_network.csv', index_col=0)
target_gene_list = []
for i, row in network_df.iterrows():
    tf_list = row.tf_list
    if pd.isnull(tf_list): 
        continue
    tf_list = tf_list.split('; ')
    tf_set = tf_set.union(set(tf_list))
    target_gene_list.append(i)


In [8]:

target_exp = pd.concat([train_target, test_target], axis=1)
source_exp = pd.concat([train_source, test_source], axis=1)
# target_gene_list = list(ref_res_df.index)
tf_list_df = pd.read_csv('../output/network_model/yeast_tf.csv', names=['tf'], index_col=0)
tf_list = list(tf_list_df.index)

X = source_exp.loc[tf_list]


In [10]:
for i in range(len(train_source_list)):
    cv_test_source = pd.concat(test_source_list, axis=1)
    cv_test_target = pd.concat(test_target_list, axis=1)
    cv_train_source_cur = []
    cv_train_target_cur = []
    for j in range(len(train_source_list)):
        if i == j: continue
        cv_train_source_cur.append(train_source_list[j])
        # cv_train_source_cur.append(test_source_list[j])
        cv_train_target_cur.append(train_target_list[j])
        # cv_train_target_cur.append(test_target_list[j])
    cv_train_source = pd.concat(cv_train_source_cur, axis=1)
    cv_train_target = pd.concat(cv_train_target_cur, axis=1)
    mp_calc = mp_run.MpCalc(target_gene_list, X, network_df, cv_train_source.loc[tf_list], cv_train_target, cv_test_source.loc[tf_list], cv_test_target)
    iter_length = len(target_gene_list)
    with Pool(cpu_count()) as p:
        r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))
    r = np.array(r)
    out_df = pd.DataFrame(index=target_gene_list)
    out_df['rf_score'] = r[:, 0]
    out_df['linear_score'] = r[:, 1]
    out_df['gs_rf_score'] = r[:, 2]
    out_df['gs_linear_score'] = r[:, 3]
    out_df['rf_with_linear_top_features_score'] = r[:, 4]
    out_df['linear_with_rf_top_features_score'] = r[:, 5]
    out_df['rf_rmse'] = r[:, 6]
    out_df['linear_rmse'] = r[:, 7]
    out_df['gs_rf_rmse'] = r[:, 8]
    out_df['gs_linear_rmse'] = r[:, 9]
    out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
    out_df['linear_with_rf_top_features_rmse'] = r[:, 11]
    out_df['rf_with_top_features_score'] = r[:, 12]
    out_df['linear_with_top_features_score'] = r[:, 13]
    out_df['rf_with_top_features_rmse'] = r[:, 14]
    out_df['linear_with_top_features_rmse'] = r[:, 15]
    out_df['rf_top_feature_num'] = r[:, 16]
    out_df['linear_top_feature_num'] = r[:, 17]
    out_df['rf_top_features_gs_overlap'] = r[:, 18]
    out_df['linear_top_features_gs_overlap'] = r[:, 19]
    out_df['rf_linear_top_features_overlap'] = r[:, 20]
    out_df['gs_edge_num'] = r[:, 21]
    out_df['test_var'] = r[:, 22]
    out_df['test_std'] = r[:, 23]
    out_df['pca_rf_score'] = r[:, 24]
    out_df['pca_rf_rmse'] = r[:, 25]
    
    print(out_df['rf_rmse'].mean())
    print(out_df['rf_score'].mean())
    print('======================================')

100%|██████████| 385/385 [01:00<00:00,  6.34it/s]


1.016153303687068
0.49977614664434294


100%|██████████| 385/385 [01:01<00:00,  6.26it/s]

0.9189593703544068
0.5657941669240302



100%|██████████| 385/385 [00:59<00:00,  6.43it/s]

0.9512252204720142
0.5681348577942763



100%|██████████| 385/385 [00:29<00:00, 13.15it/s]


2.2363093595334176
-1.0321252441167548


In [9]:
for i in range(len(train_source_list)):
    cv_test_source = pd.concat([train_source_list[i], test_source_list[i]], axis=1)
    cv_test_target = pd.concat([train_target_list[i], test_target_list[i]], axis=1)
    cv_train_source_cur = []
    cv_train_target_cur = []
    for j in range(len(train_source_list)):
        if i == j: continue
        cv_train_source_cur.append(train_source_list[j])
        cv_train_source_cur.append(test_source_list[j])
        cv_train_target_cur.append(train_target_list[j])
        cv_train_target_cur.append(test_target_list[j])
    cv_train_source = pd.concat(cv_train_source_cur, axis=1)
    cv_train_target = pd.concat(cv_train_target_cur, axis=1)
    mp_calc = mp_run.MpCalc(target_gene_list, X, network_df, cv_train_source.loc[tf_list], cv_train_target, cv_test_source.loc[tf_list], cv_test_target)
    iter_length = len(target_gene_list)
    with Pool(cpu_count()) as p:
        r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))
    r = np.array(r)
    out_df = pd.DataFrame(index=target_gene_list)
    out_df['rf_score'] = r[:, 0]
    out_df['linear_score'] = r[:, 1]
    out_df['gs_rf_score'] = r[:, 2]
    out_df['gs_linear_score'] = r[:, 3]
    out_df['rf_with_linear_top_features_score'] = r[:, 4]
    out_df['linear_with_rf_top_features_score'] = r[:, 5]
    out_df['rf_rmse'] = r[:, 6]
    out_df['linear_rmse'] = r[:, 7]
    out_df['gs_rf_rmse'] = r[:, 8]
    out_df['gs_linear_rmse'] = r[:, 9]
    out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
    out_df['linear_with_rf_top_features_rmse'] = r[:, 11]
    out_df['rf_with_top_features_score'] = r[:, 12]
    out_df['linear_with_top_features_score'] = r[:, 13]
    out_df['rf_with_top_features_rmse'] = r[:, 14]
    out_df['linear_with_top_features_rmse'] = r[:, 15]
    out_df['rf_top_feature_num'] = r[:, 16]
    out_df['linear_top_feature_num'] = r[:, 17]
    out_df['rf_top_features_gs_overlap'] = r[:, 18]
    out_df['linear_top_features_gs_overlap'] = r[:, 19]
    out_df['rf_linear_top_features_overlap'] = r[:, 20]
    out_df['gs_edge_num'] = r[:, 21]
    out_df['test_var'] = r[:, 22]
    out_df['test_std'] = r[:, 23]
    out_df['pca_rf_score'] = r[:, 24]
    out_df['pca_rf_rmse'] = r[:, 25]
    
    print(out_df['rf_rmse'].mean())
    print(out_df['rf_score'].mean())
    print('======================================')

100%|██████████| 385/385 [01:11<00:00,  5.37it/s]

1.8458953177774877
-423.8226995061593



100%|██████████| 385/385 [01:11<00:00,  5.37it/s]

1.703050056756258
-2052.9873622016617



100%|██████████| 385/385 [01:13<00:00,  5.25it/s]


0.9596913215567191
-1539.2940390025374


100%|██████████| 385/385 [00:41<00:00,  9.30it/s]

2.3759392638077674
-79.74738779250069


In [12]:
for i in range(len(train_source_list)):
    cv_test_source = pd.concat(alt_test_source_list, axis=1)
    cv_test_target = pd.concat(alt_test_target_list, axis=1)
    cv_train_source_cur = []
    cv_train_target_cur = []
    for j in range(len(train_source_list)):
        if i == j: continue
        cv_train_source_cur.append(alt_train_source_list[j])
        # cv_train_source_cur.append(test_source_list[j])
        cv_train_target_cur.append(alt_train_target_list[j])
        # cv_train_target_cur.append(test_target_list[j])
    cv_train_source = pd.concat(cv_train_source_cur, axis=1)
    cv_train_target = pd.concat(cv_train_target_cur, axis=1)
    mp_calc = mp_run.MpCalc(target_gene_list, X, network_df, cv_train_source.loc[tf_list], cv_train_target, cv_test_source.loc[tf_list], cv_test_target)
    iter_length = len(target_gene_list)
    with Pool(cpu_count()) as p:
        r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))
    r = np.array(r)
    out_df = pd.DataFrame(index=target_gene_list)
    out_df['rf_score'] = r[:, 0]
    out_df['linear_score'] = r[:, 1]
    out_df['gs_rf_score'] = r[:, 2]
    out_df['gs_linear_score'] = r[:, 3]
    out_df['rf_with_linear_top_features_score'] = r[:, 4]
    out_df['linear_with_rf_top_features_score'] = r[:, 5]
    out_df['rf_rmse'] = r[:, 6]
    out_df['linear_rmse'] = r[:, 7]
    out_df['gs_rf_rmse'] = r[:, 8]
    out_df['gs_linear_rmse'] = r[:, 9]
    out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
    out_df['linear_with_rf_top_features_rmse'] = r[:, 11]
    out_df['rf_with_top_features_score'] = r[:, 12]
    out_df['linear_with_top_features_score'] = r[:, 13]
    out_df['rf_with_top_features_rmse'] = r[:, 14]
    out_df['linear_with_top_features_rmse'] = r[:, 15]
    out_df['rf_top_feature_num'] = r[:, 16]
    out_df['linear_top_feature_num'] = r[:, 17]
    out_df['rf_top_features_gs_overlap'] = r[:, 18]
    out_df['linear_top_features_gs_overlap'] = r[:, 19]
    out_df['rf_linear_top_features_overlap'] = r[:, 20]
    out_df['gs_edge_num'] = r[:, 21]
    out_df['test_var'] = r[:, 22]
    out_df['test_std'] = r[:, 23]
    out_df['pca_rf_score'] = r[:, 24]
    out_df['pca_rf_rmse'] = r[:, 25]
    
    print(out_df['rf_rmse'].mean())
    print(out_df['rf_score'].mean())
    print('======================================')

100%|██████████| 385/385 [00:51<00:00,  7.44it/s]

0.8353963898034658
0.454305597839694



100%|██████████| 385/385 [00:51<00:00,  7.48it/s]

1.1228040385281808
0.0324687494721939



100%|██████████| 385/385 [00:52<00:00,  7.34it/s]

0.8826593326947497
0.4103427122851364



100%|██████████| 385/385 [00:44<00:00,  8.65it/s]

0.9075994981802961
0.32510776302733746


In [11]:
for i in range(len(train_source_list)):
    cv_test_source = pd.concat([alt_train_source_list[i], alt_test_source_list[i]], axis=1)
    cv_test_target = pd.concat([alt_train_target_list[i], alt_test_target_list[i]], axis=1)
    cv_train_source_cur = []
    cv_train_target_cur = []
    for j in range(len(train_source_list)):
        if i == j: continue
        cv_train_source_cur.append(alt_train_source_list[j])
        cv_train_source_cur.append(alt_test_source_list[j])
        cv_train_target_cur.append(alt_train_target_list[j])
        cv_train_target_cur.append(alt_test_target_list[j])
    cv_train_source = pd.concat(cv_train_source_cur, axis=1)
    cv_train_target = pd.concat(cv_train_target_cur, axis=1)
    mp_calc = mp_run.MpCalc(target_gene_list, X, network_df, cv_train_source.loc[tf_list], cv_train_target, cv_test_source.loc[tf_list], cv_test_target)
    iter_length = len(target_gene_list)
    with Pool(cpu_count()) as p:
        r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))
    r = np.array(r)
    out_df = pd.DataFrame(index=target_gene_list)
    out_df['rf_score'] = r[:, 0]
    out_df['linear_score'] = r[:, 1]
    out_df['gs_rf_score'] = r[:, 2]
    out_df['gs_linear_score'] = r[:, 3]
    out_df['rf_with_linear_top_features_score'] = r[:, 4]
    out_df['linear_with_rf_top_features_score'] = r[:, 5]
    out_df['rf_rmse'] = r[:, 6]
    out_df['linear_rmse'] = r[:, 7]
    out_df['gs_rf_rmse'] = r[:, 8]
    out_df['gs_linear_rmse'] = r[:, 9]
    out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
    out_df['linear_with_rf_top_features_rmse'] = r[:, 11]
    out_df['rf_with_top_features_score'] = r[:, 12]
    out_df['linear_with_top_features_score'] = r[:, 13]
    out_df['rf_with_top_features_rmse'] = r[:, 14]
    out_df['linear_with_top_features_rmse'] = r[:, 15]
    out_df['rf_top_feature_num'] = r[:, 16]
    out_df['linear_top_feature_num'] = r[:, 17]
    out_df['rf_top_features_gs_overlap'] = r[:, 18]
    out_df['linear_top_features_gs_overlap'] = r[:, 19]
    out_df['rf_linear_top_features_overlap'] = r[:, 20]
    out_df['gs_edge_num'] = r[:, 21]
    out_df['test_var'] = r[:, 22]
    out_df['test_std'] = r[:, 23]
    out_df['pca_rf_score'] = r[:, 24]
    out_df['pca_rf_rmse'] = r[:, 25]
    
    print(out_df['rf_rmse'].mean())
    print(out_df['rf_score'].mean())
    print('======================================')

100%|██████████| 385/385 [01:02<00:00,  6.17it/s]

1.1594210226249606
-202.09911406737874



100%|██████████| 385/385 [01:01<00:00,  6.30it/s]

2.315068291843149
-4483.2036138157455



100%|██████████| 385/385 [00:58<00:00,  6.59it/s]

1.1564569503234166
-7.820311365500248



100%|██████████| 385/385 [00:49<00:00,  7.71it/s]

0.710998356433654
-35.864435407290685


In [9]:
for i in range(len(train_source_list)):
    cv_test_source = alt_test_source_list[i]
    cv_test_target = alt_test_target_list[i]
    cv_train_source = alt_train_source_list[i]
    cv_train_target = alt_train_target_list[i]
    mp_calc = mp_run.MpCalc(target_gene_list, X, network_df, cv_train_source.loc[tf_list], cv_train_target, cv_test_source.loc[tf_list], cv_test_target)
    iter_length = len(target_gene_list)
    with Pool(cpu_count()) as p:
        r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))
    r = np.array(r)
    out_df = pd.DataFrame(index=target_gene_list)
    out_df['rf_score'] = r[:, 0]
    out_df['linear_score'] = r[:, 1]
    out_df['gs_rf_score'] = r[:, 2]
    out_df['gs_linear_score'] = r[:, 3]
    out_df['rf_with_linear_top_features_score'] = r[:, 4]
    out_df['linear_with_rf_top_features_score'] = r[:, 5]
    out_df['rf_rmse'] = r[:, 6]
    out_df['linear_rmse'] = r[:, 7]
    out_df['gs_rf_rmse'] = r[:, 8]
    out_df['gs_linear_rmse'] = r[:, 9]
    out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
    out_df['linear_with_rf_top_features_rmse'] = r[:, 11]
    out_df['rf_with_top_features_score'] = r[:, 12]
    out_df['linear_with_top_features_score'] = r[:, 13]
    out_df['rf_with_top_features_rmse'] = r[:, 14]
    out_df['linear_with_top_features_rmse'] = r[:, 15]
    out_df['rf_top_feature_num'] = r[:, 16]
    out_df['linear_top_feature_num'] = r[:, 17]
    out_df['rf_top_features_gs_overlap'] = r[:, 18]
    out_df['linear_top_features_gs_overlap'] = r[:, 19]
    out_df['rf_linear_top_features_overlap'] = r[:, 20]
    out_df['gs_edge_num'] = r[:, 21]
    out_df['test_var'] = r[:, 22]
    out_df['test_std'] = r[:, 23]
    out_df['pca_rf_score'] = r[:, 24]
    out_df['pca_rf_rmse'] = r[:, 25]
    
    print(out_df['rf_rmse'].mean())
    print(out_df['rf_score'].mean())
    print('======================================')

100%|██████████| 385/385 [00:40<00:00,  9.53it/s]

0.5919961441812833
-14.325552789434239



100%|██████████| 385/385 [00:39<00:00,  9.75it/s]

0.3961666052112541
-2.9916133801109086



100%|██████████| 385/385 [00:41<00:00,  9.34it/s]

0.7307676006144243
-0.8795869843210535



100%|██████████| 385/385 [00:36<00:00, 10.54it/s]


0.6763618757503286
-0.7223858164775937


In [12]:
all_rmse_list = []
all_train_rmse_list = []
all_train_source = pd.concat(train_source_list, axis=1)
all_train_target = pd.concat(train_target_list, axis=1)
for i in range(len(train_source_list)):
    cv_test_source = test_source_list[i]
    cv_test_target = test_target_list[i]
    cv_train_source = train_source_list[i]
    cv_train_target = train_target_list[i]
    new_test_target = cv_test_target
    new_test_target = new_test_target.loc[new_test_target.std(axis=1) > 0.5]
    new_target_gene_list = new_test_target.index
    mp_calc = mp_run.MpCalc(new_target_gene_list, X, network_df, cv_train_source.loc[tf_list], cv_train_target, cv_test_source.loc[tf_list], cv_test_target)
    iter_length = len(new_target_gene_list)
    with Pool(cpu_count()) as p:
        r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))
    r = np.array(r)
    out_df = pd.DataFrame(index=new_target_gene_list)
    out_df['rf_score'] = r[:, 0]
    out_df['linear_score'] = r[:, 1]
    out_df['gs_rf_score'] = r[:, 2]
    out_df['gs_linear_score'] = r[:, 3]
    out_df['rf_with_linear_top_features_score'] = r[:, 4]
    out_df['linear_with_rf_top_features_score'] = r[:, 5]
    out_df['rf_rmse'] = r[:, 6]
    out_df['linear_rmse'] = r[:, 7]
    out_df['gs_rf_rmse'] = r[:, 8]
    out_df['gs_linear_rmse'] = r[:, 9]
    out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
    out_df['linear_with_rf_top_features_rmse'] = r[:, 11]
    out_df['rf_with_top_features_score'] = r[:, 12]
    out_df['linear_with_top_features_score'] = r[:, 13]
    out_df['rf_with_top_features_rmse'] = r[:, 14]
    out_df['linear_with_top_features_rmse'] = r[:, 15]
    out_df['rf_top_feature_num'] = r[:, 16]
    out_df['linear_top_feature_num'] = r[:, 17]
    out_df['rf_top_features_gs_overlap'] = r[:, 18]
    out_df['linear_top_features_gs_overlap'] = r[:, 19]
    out_df['rf_linear_top_features_overlap'] = r[:, 20]
    out_df['gs_edge_num'] = r[:, 21]
    out_df['test_var'] = r[:, 22]
    out_df['test_std'] = r[:, 23]
    out_df['pca_rf_score'] = r[:, 24]
    out_df['pca_rf_rmse'] = r[:, 25]
    
    all_rmse_list = np.concatenate([all_rmse_list, r[:, 6]])
    print(out_df['rf_rmse'].mean())
    print(out_df['rf_score'].mean())

    mp_calc = mp_run.MpCalc(new_target_gene_list, X, network_df, all_train_source.loc[tf_list], all_train_target, cv_test_source.loc[tf_list], cv_test_target)
    iter_length = len(new_target_gene_list)
    with Pool(cpu_count()) as p:
        r = list(tqdm(p.imap(mp_calc.full_comp_new, range(iter_length)), total=iter_length))
    r = np.array(r)
    print(np.mean(r[:, 6]))
    all_train_rmse_list = np.concatenate([all_train_rmse_list, r[:, 6]])


    print('======================================')
print(np.mean(all_rmse_list))
print(np.mean(all_train_rmse_list))

100%|██████████| 213/213 [00:16<00:00, 13.12it/s]

1.0325858957456335
0.11036959068452429



100%|██████████| 213/213 [00:34<00:00,  6.22it/s]

0.9924473786674606



100%|██████████| 94/94 [00:05<00:00, 17.36it/s]


...

1.1501166901522593


100%|██████████| 310/310 [00:46<00:00,  6.60it/s]

0.9279654147260404
0.5376763964404639



100%|██████████| 310/310 [00:53<00:00,  5.85it/s]

0.9235187559959913
1.3173980357263575
1.3056782051342815


In [17]:
# sanity check:
for i in train_source_list:
    for j in test_source_list:
        print(set(i.columns).intersection(set(j.columns)))

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [16]:

for i in train_target_list:
    for j in test_target_list:
        print(set(i.columns).intersection(set(j.columns)))

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
